
## Compute the associated average outward flux

To have a good quality gradient of the distance map, perform a light smooth over it. Define convolution kernels in each direction and use them recursively. 

In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from vmtk import vmtkscripts
from vmtk import vtkvmtk
import timeit
import os
import sys
import numpy as np
from vtk.numpy_interface import dataset_adapter as dsa
import random
import networkx as nx
import vtk
from joblib import Parallel, delayed
import math

from vtkdatawidgets.vtk_binding import VtkJupyterBridge

bridge = VtkJupyterBridge()
sphere = vtk.vtkSphereSource()
bridge.SetInputConnection(sphere.GetOutputPort())
bridge.Update()

imgr = vmtkscripts.vmtkImageReader()
imgr.InputFileName = os.path.join(os.getcwd(), 'test-surf-normal-hole.mha')
imgr.Execute()

mc = vmtkscripts.vmtkMarchingCubes()
mc.Image = imgr.Image
mc.Level = 1000.0
mc.Execute()

In [2]:
polyData = vtk.vtkPolyData()

In [5]:
surfr = vmtkscripts.vmtkSurfaceReader()
surfr.InputFileName = os.path.join(os.getcwd(), 'test-surf-no-hole.stl')
surfr.Execute()

surf = surfr.Surface

Reading STL surface file.


In [3]:
view = vmtkscripts.vmtkSurfaceViewer()
view.Surface = surf
view.Execute()

Quit renderer


In [6]:
cl = vmtkscripts.vmtkCenterlinesNetwork()
cl.Surface = surf
cl.Execute()

Progress: 100%Progress: 100%
Cleaning surface.
Triangulating surface.
Computing Centerlines ...
Cleaning PolyData
Stripping PolyData


In [4]:
cl = vmtkscripts.vmtkCenterlines()
cl.Surface = surf
cl.Execute()

Cleaning surface.
Triangulating surface.
Capping surface.
Please position the mouse and press space to add source points, 'u' to undo
Quit renderer
Please position the mouse and press space to add target points, 'u' to undo
Quit renderer
Computing centerlines.
Computing centerlines...

In [8]:
write = vmtkscripts.vmtkSurfaceWriter()
write.Surface = cl.Centerlines
write.OutputFileName = os.path.join(os.getcwd(), 'cldata', 'centerlines.vtp')
write.Execute()

Writing VTK XML surface file.


In [5]:
print(cl.OrigCenterlines)

vtkPolyData (0x7ff24441bbe0)
  Debug: Off
  Modified Time: 12751199
  Reference Count: 1
  Registered Events: (none)
  Information: 0x7ff24441bd50
  Data Released: False
  Global Release Data: Off
  UpdateTime: 12751365
  Field Data:
    Debug: Off
    Modified Time: 12751106
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 2265
  Number Of Cells: 8
  Cell Data:
    Debug: Off
    Modified Time: 12751114
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x7ff24441c2a0)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x7ff24441bfd0
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    Vectors: (none)
    Nor

In [5]:
view2 = vmtkscripts.vmtkCenterlineViewer()
view2.Centerlines = cl.Centerlines
view2.PointDataArrayName = 'MaximumInscribedSphereRadius'
view2.Execute()

Quit renderer


In [5]:
conv = vmtkscripts.vmtkCenterlinesToNumpy()
conv.Centerlines = cl.Centerlines
conv.Execute()

wrapping vtkPolyData object
converting cell data: 
converting points
converting point data: 
MaximumInscribedSphereRadius
EdgeArray
EdgePCoordArray
converting cell connectivity list


In [6]:
ad = conv.ArrayDict

In [7]:
ad

{'Points': array([[ 14.98812103,   5.29034901,  21.58057594],
        [ 14.96139908,   5.31326914,  21.5819149 ],
        [ 14.90267467,   5.35814762,  21.59190941],
        ..., 
        [ 28.92583656,   8.2573843 ,  -7.74222136],
        [ 28.92563057,   8.25743008,  -7.74235201],
        [ 28.90869904,   8.26122189,  -7.75310802]], dtype=float32),
 'PointData': {'MaximumInscribedSphereRadius': array([ 0.61773737,  0.61816763,  0.61757536, ...,  1.10918818,
          1.10905313,  1.09793406]), 'EdgeArray': array([[ 56332,  56332],
         [ 56384,  56385],
         [ 56378,  56378],
         ..., 
         [175339, 175339],
         [173452, 175339],
         [173452, 173452]], dtype=int32), 'EdgePCoordArray': array([ 0.   ,  0.02 ,  0.   , ...,  0.   ,  0.988,  0.   ])},
 'CellData': {'CellPointIds': [array([1501, 1500, 1499, ...,    2,    1,    0], dtype=int32),
   array([2651, 2650, 2649, ..., 1504, 1503, 1502], dtype=int32),
   array([3656, 3655, 3654, ..., 2654, 2653, 2652], dt

In [8]:
pd = vtk.vtkPolyData()
nppd = dsa.WrapDataObject(pd)

In [9]:
points = ad['Points']
vtkPoints = dsa.numpyTovtkDataArray(points, 'Points')

In [12]:
nppd.Points = vtkPoints

In [14]:
for pointDataKeys in ad['PointData'].keys():
    pointDataItem = ad['PointData'][pointDataKeys]
    nppd.PointData.append(pointDataItem, name=pointDataKeys)

In [16]:
for cellKey in ad['CellData'].keys():
    
    if cellKey == 'CellPointIds':
        cellDataArray = vtk.vtkCellArray()
        for cellId in ad['CellData']['CellPointIds']:
            numberOfCellPoints = cellId.size
            cellDataArray.InsertNextCell(numberOfCellPoints)
            for cellPoint in cellId:
                cellDataArray.InsertCellPoint(cellPoint)

        nppd.VTKObject.SetLines(cellDataArray)
    else:
        cellDataItem = ad['CellData'][cellKey]
        nppd.CellData.append(cellDataItem, name='cellKey')

In [19]:
outarry = nppd.VTKObject

In [22]:
viewer = vmtkscripts.vmtkCenterlineViewer()
viewer.Centerlines = outarry
viewer.PointDataArrayName = 'EdgeArray'
viewer.Execute()

Quit renderer


In [ ]:
pd = vtk.vtkPolyDa

In [11]:
rand = np.zeros(shape=npcl.PointData['EdgeArray'].shape)

In [12]:
pdar = vtk.vtkPointData()

In [16]:
vtrands = dsa.numpyTovtkDataArray(rand, name='RandEdgeArray')

In [19]:
print(vtrands)

vtkDoubleArray (0x7fb4df60c8a0)
  Debug: Off
  Modified Time: 16079109
  Reference Count: 1
  Registered Events: (none)
  Name: RandEdgeArray
  Data type: double
  Size: 7424
  MaxId: 7423
  NumberOfComponents: 2
  Information: 0x7fb4df606c20
    Debug: Off
    Modified Time: 16079108
    Reference Count: 1
    Registered Events: (none)
  Name: RandEdgeArray
  Number Of Components: 2
  Number Of Tuples: 3712
  Size: 7424
  MaxId: 7423
  LookupTable: (none)




In [21]:
npcl.PointData.append(rand, name='RandEdgeArray')

In [23]:
npcl.PointData.keys()

['MaximumInscribedSphereRadius',
 'EdgeArray',
 'EdgePCoordArray',
 'RandEdgeArray']

In [ ]:
cent.get

In [ ]:
a = dsa.WrapDataObject(cent)

In [ ]:
a.PointData['EdgeArray']

In [ ]:
fedges = vtk.vtkFeatureEdges()
fedges.BoundaryEdgesOn()
fedges.FeatureEdgesOff()
fedges.ManifoldEdgesOff()
fedges.SetInputData(surf)
fedges.Update()

ofedges = fedges.GetOutput()

numEdges = ofedges.GetNumberOfPoints()

In [ ]:
if numEdges != 0:
    tempcapper = vmtkscripts.vmtkSurfaceCapper()
    tempcapper.Surface = surf
    tempcapper.Interactive = 0
    tempcapper.Execute()
    
    networkSurface = tempcapper.Surface
else:
    networkSurface = surf

In [ ]:
numCells = surf.GetNumberOfCells()
cellToDelete = random.randrange(0, numCells-1)

networkSurface.BuildLinks()
networkSurface.DeleteCell(cellToDelete)
networkSurface.RemoveDeletedCells()

net = vmtkscripts.vmtkNetworkExtraction()
net.Surface = networkSurface
net.AdvancementRatio = 1.001
net.Execute()

network = net.Network

In [ ]:
convert = vmtkscripts.vmtkCenterlinesToNumpy()
convert.Centerlines = network
convert.Execute()
ad = convert.ArrayDict

In [ ]:
cellDataTopology = ad['CellData']['Topology']

nodeIndexToIgnore = np.where(cellDataTopology[:,0] == 0)[0][0]

keepCellConnectivityList = []
pointIdxToKeep = np.array([])
for loopIdx, cellConnectivityList in enumerate(ad['CellData']['CellPointIds']):
    if loopIdx == nodeIndexToIgnore:
        removeCellStartIdx = cellConnectivityList[0]
        removeCellEndIdx = cellConnectivityList[-1]
        removeCellLength = cellConnectivityList.size
        if (removeCellEndIdx + 1) - removeCellStartIdx != removeCellLength:
            raise(ValueError)
        continue
    else:
        rescaledCellConnectivity = np.subtract(cellConnectivityList, removeCellLength, where=cellConnectivityList >= removeCellLength)
        keepCellConnectivityList.append(rescaledCellConnectivity)
        pointIdxToKeep = np.concatenate((pointIdxToKeep, cellConnectivityList)).astype(np.int)
        

newPoints = ad['Points'][pointIdxToKeep]

newRadius = ad['PointData']['Radius'][pointIdxToKeep]

In [ ]:
def compute_centerlines(surfaceAddress, delaunayAddress, cell, points):
    '''a method to compute centerlines which can be called in parallel
    
    Arguments:
        surface (obj:`vtkPolyData`): the input surface to calculate centerlines of
        delaunay (obj:`vtkUnstructuredGrid`): a previously computed delaunay triangulation
            of the surface
        cell (np.array): the cellID connectivity list
        points (np.array): the x,y,z coordinates of points identified in the cell argument
    '''
    
    cellStartIdx = cell[0]
    cellEndIdx = cell[-1]
    cellStartPoint = points[cellStartIdx].tolist()
    cellEndPoint = points[cellEndIdx].tolist()
    
    surface = vtk.vtkPolyData(surfaceAddress)
    delaunay = vtk.vtkUnstructuredGrid(delaunayAddress)
    
    cl = vmtkscripts.vmtkCenterlines()
    cl.Surface = surface
    cl.DelaunayTessellation = delaunay
    cl.SeedSelectorName = 'pointlist'
    cl.SourcePoints = cellStartPoint
    cl.TargetPoints = cellEndPoint
    cl.Execute()
    
    clConvert = vmtkscripts.vmtkCenterlinesToNumpy()
    clConvert.Centerlines = cl.Centerlines
    clConvert.Execute()
    
    return clConvert.ArrayDict

In [ ]:
tessalation = vmtkscripts.vmtkDelaunayVoronoi()
tessalation.Surface = networkSurface
tessalation.Execute()
delaunay = tessalation.DelaunayTessellation

In [ ]:
networkSurfaceMemoryAddress = networkSurface.__this__
delaunayMemoryAddress = delaunay.__this__

In [ ]:
outlist = Parallel(n_jobs=-1, backend='multiprocessing', verbose=10)(
    delayed(compute_centerlines)(networkSurfaceMemoryAddress,
                                 delaunayMemoryAddress,
                                 cell,
                                 newPoints) for cell in keepCellConnectivityList)

In [ ]:
npConvert = vmtkscripts.vmtkNumpyToCenterlines()
npConvert.ArrayDict = outlist[0]
npConvert.Execute()

In [ ]:
outlist[0]['PointData']['EdgeArray']

In [ ]:
outlist[0]['Points'].shape

In [ ]:
print(npConvert.Centerlines)

In [ ]:
npConvert.ArrayDict['PointData']

In [ ]:
out = []
for item in outlist:
    npConvert = vmtkscripts.vmtkNumpyToCenterlines()
    npConvert.ArrayDict = item
    npConvert.Execute()
    out.append(npConvert.Centerlines)

In [ ]:
new = vtk.vtkAppendPolyData()
for data in out:  
    new.AddInputData(data)
new.Update()

combined = new.GetOutput()

In [ ]:
sv = vmtkscripts.vmtkSurfaceViewer()
sv.Surface = b
sv.Execute()

In [ ]:
cv = vmtkscripts.vmtkCenterlineViewer()
cv.Centerlines = combined
cv.PointDataArrayName = 'MaximumInscribedSphereRadius'
cv.Execute()

In [ ]:
convert2 = vmtkscripts.vmtkCenterlinesToNumpy()
convert2.Centerlines = cl.Centerlines
convert2.Execute()
ad2 = convert2.ArrayDict

In [ ]:
ad2

In [6]:
rend = vmtkscripts.vmtkRenderer()
rend.Execute()

view = vmtkscripts.vmtkCenterlineViewer()
view.vmtkRenderer = rend.vmtkRenderer
view.Centerlines = cl.OrigCenterlines
view.Execute()

view2 = vmtkscripts.vmtkCenterlineViewer()
view2.vmtkRenderer = rend.vmtkRenderer
view2.Centerlines = cl.Centerlines
view2.PointDataArrayName = 'MaximumInscribedSphereRadius'
view2.Execute()

Quit renderer
Quit renderer


In [ ]:
write = vmtkscripts.vmtkSurfaceWriter()
write.Surface = net.Network
write.OutputFileName = os.path.join(os.getcwd(), 'test-multi-hole.vtp')
write.Execute()